In [33]:
import requests
import json
import facebook
import re
from datetime import datetime
import concurrent.futures
from pymongo import MongoClient

In [34]:
# fan_page = "DoctorKoWJ"

token = input("your token")
group_id = requests.get('https://graph.facebook.com/?id={}&access_token={}'.format("136845026417486", token)).json()["id"]
graph = facebook.GraphAPI(access_token = token)

client = MongoClient("mongodb://192.168.75.130:27017")
db = client["fb"]

In [35]:
def get_comments(post_id):
    comments = []
    obj_page = requests.get("https://graph.facebook.com/v2.6/%s?fields=comments{created_time,id,message,likes.limit(0).summary(True),from}&access_token=%s"%(post_id,token)).json()["comments"]
    
    
    while True:
        for comment in obj_page["data"]:
            comment["created_time"] = datetime.strptime(comment["created_time"], "%Y-%m-%dT%H:%M:%S+0000")
            comment["likes"] = comment["likes"]["summary"]["total_count"]
            comment["post_id"] = post_id
            comments.append(comment)
            print("  finish {}".format(comment["id"]))

        try:
            if "next" in obj_page["paging"]:
                next_page = obj_page["paging"]["next"]
                obj_page = requests.get(next_page).json()
            else:
                break
        except:
            break
        
    return comments

In [36]:
def get_posts():
    kp_posts = []
    obj_page = graph.request(group_id,{'fields':'posts.since(2017-01-01){created_time, message,id,likes.limit(0).summary(True),shares,link}'})["posts"]

    while True:
        for post in obj_page["data"]:
            post["created_time"] = datetime.strptime(post["created_time"], "%Y-%m-%dT%H:%M:%S+0000")
            post["likes"] = post["likes"]["summary"]["total_count"]
            if "shares" in post:
                post["shares"] = post["shares"]["count"]
            else:
                post["shares"] = 0
            kp_posts.append(post)
            
            print("finish {}".format(post["id"]))

        if "next" in obj_page["paging"]:
            next_page = obj_page["paging"]["next"]
            obj_page = requests.get(next_page).json()
        else:
            break
    
    return kp_posts

In [37]:
def main():
    kp_posts = get_posts()
    posts_collections = db["posts"]
    posts_collections.insert_many(kp_posts)
    post_ids = [post["id"] for post in kp_posts]
    
    kp_comments_collections = db["kp_comments"]
    for post_id in post_ids:
        comments = get_comments(post_id)
        kp_comments_collections.insert_many(comments)

In [ ]:
if __name__ == "__main__":
    main()